In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
!pip install catboost

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 76.6/76.6 MB 12.0 MB/s eta 0:00:00


In [ ]:
pip install optuna

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 365.3/365.3 KB 24.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 210.5/210.5 KB 22.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 78.7/78.7 KB 10.9 MB/s eta 0:00:00


In [ ]:
import warnings
warnings.filterwarnings('ignore')

import os
import random
from tqdm import tqdm
import pandas as pd
import numpy as np

from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import RepeatedStratifiedKFold
from sklearn.metrics import f1_score
from imblearn.over_sampling import BorderlineSMOTE
from sklearn.decomposition import PCA
from sklearn.preprocessing import StandardScaler
from sklearn.inspection import permutation_importance
import matplotlib.pyplot as plt
import catboost

def seed_everything(seed):
    random.seed(seed)
    os.environ['PYTHONHASHSEED'] = str(seed)
    np.random.seed(seed)
seed_everything(37) # Seed 고정

In [ ]:
import sklearn

In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder

train_df = pd.read_csv('/content/drive/MyDrive/open/train.csv')
test_df = pd.read_csv('/content/drive/MyDrive/open/test.csv')
print(len(train_df.columns))

#쓰레기 변수 지우기
a=[]
A = pd.DataFrame(train_df.isnull().sum(),columns=['sum'])
a = list(A[A['sum']==train_df.shape[0]].index)
train_df.drop(columns=a,inplace=True)
a=[]
print(len(train_df.columns))

from datetime import datetime
month =[]
day = []
hour =[]
minute = []
for i in range(train_df.shape[0]):
  month.append(datetime.strptime(train_df.iloc[i,:]['TIMESTAMP'],'%Y-%m-%d %H:%M').month)
  day.append(datetime.strptime(train_df.iloc[i,:]['TIMESTAMP'],'%Y-%m-%d %H:%M').day)
  hour.append(datetime.strptime(train_df.iloc[i,:]['TIMESTAMP'],'%Y-%m-%d %H:%M').hour)
  minute.append(datetime.strptime(train_df.iloc[i,:]['TIMESTAMP'],'%Y-%m-%d %H:%M').minute)

train_df['MONTH'] = month
train_df['DAY'] = day
train_df['HOUR'] = hour
train_df['MINUTE'] = minute

train_x = train_df.drop(columns=['PRODUCT_ID', 'TIMESTAMP'])
train_x = train_x.fillna(0)

qual_col = ['LINE', 'PRODUCT_CODE']

for i in qual_col:
    le = LabelEncoder()
    le = le.fit(train_x[i])
    train_x[i] = le.transform(train_x[i])
print('Done.')

train_y = train_x['Y_Quality']
train_yy = train_x['Y_Class']
train_x.drop(columns=['Y_Quality','Y_Class'],inplace=True)

a = train_df[['Y_Class','Y_Quality']].groupby('Y_Class').agg(['mean', 'min', 'max', 'count'])

2881
2799
Done.


In [ ]:
def cb_objective(trial):

    params = {
            'bootstrap_type':trial.suggest_categorical('bootstrap_type', ['Bayesian', 'Bernoulli']),
            'od_wait':trial.suggest_int('od_wait', 500, 1000),
            'learning_rate' : trial.suggest_uniform('learning_rate',0.01,0.1),
            'reg_lambda': trial.suggest_uniform('reg_lambda',1e-5,100),
            'random_strength': trial.suggest_uniform('random_strength',0,100),
            #'depth': trial.suggest_int('depth',2,15),
            'min_data_in_leaf': trial.suggest_int('min_data_in_leaf',1,20),
            'leaf_estimation_iterations': trial.suggest_int('leaf_estimation_iterations',1,15),
            'verbose': False,
            #"eval_metric": 'TotalF1',
            "one_hot_max_size": trial.suggest_int("one_hot_max_size", 1, 5),
            'task_type': 'GPU',
        }

    reg = catboost.CatBoostRegressor(**params)

    rskfold = RepeatedStratifiedKFold(n_splits=5, n_repeats=3,random_state = 37)

    f1_list = []
    for fold, (train_index, val_index) in enumerate(tqdm(rskfold.split(train_x, train_yy))):
        x_trn, x_val, yy_trn, yy_val = train_x.loc[train_index], train_x.loc[val_index], train_yy.loc[train_index], train_yy.loc[val_index]
        y_trn = train_y.loc[train_index]
        y_val = train_y.loc[val_index]

        reg.fit(x_trn, y_trn, early_stopping_rounds=100, cat_features=['PRODUCT_CODE', 'LINE'])
        pre_preds = reg.predict(x_val)

        preds=[]
        for p in pre_preds:
          if p<=a[('Y_Quality','max')][0]:
            preds.append(0)
          elif p<=a[('Y_Quality','min')][2]:
            preds.append(1)
          else:
            preds.append(2)
        f1_list.append(f1_score(yy_val, preds, average='macro' ))

    return np.mean(f1_list)

In [ ]:
import optuna

# study = optuna.create_study(direction='maximize')
# study.optimize(cb_objective, n_trials=20, show_progress_bar=True)

study = optuna.create_study(direction='maximize')
study.optimize(cb_objective, n_trials=20, show_progress_bar=True)

[I 2023-02-25 15:01:13,136] A new study created in memory with name: no-name-0d743559-1df6-42eb-8a37-279b4f9472dc


  0%|          | 0/20 [00:00<?, ?it/s]


0it [00:00, ?it/s]
1it [01:40, 100.15s/it]
2it [03:20, 100.24s/it]
3it [04:59, 99.52s/it] 
4it [06:36, 98.72s/it]
5it [08:14, 98.57s/it]
6it [09:52, 98.29s/it]
7it [11:30, 98.26s/it]
8it [13:07, 97.83s/it]
9it [14:46, 98.16s/it]
10it [16:24, 98.16s/it]
11it [18:05, 98.98s/it]
12it [19:43, 98.73s/it]
13it [21:21, 98.31s/it]
14it [22:58, 98.14s/it]
15it [24:38, 98.56s/it]


[I 2023-02-25 15:25:51,557] Trial 0 finished with value: 0.6553224320563107 and parameters: {'bootstrap_type': 'Bernoulli', 'od_wait': 508, 'learning_rate': 0.046753743851373035, 'reg_lambda': 85.10503877560288, 'random_strength': 57.27570569615402, 'min_data_in_leaf': 2, 'leaf_estimation_iterations': 14, 'one_hot_max_size': 2}. Best is trial 0 with value: 0.6553224320563107.



0it [00:00, ?it/s]
1it [01:29, 89.53s/it]
2it [02:57, 88.77s/it]
3it [04:26, 88.56s/it]

In [ ]:
fig = optuna.visualization.plot_optimization_history(study)
fig.show()

In [ ]:
fig = optuna.visualization.plot_parallel_coordinate(study)
fig.show()

In [ ]:
fig = optuna.visualization.plot_param_importances(study)
fig.show()

In [ ]:
print(study.best_trial.value, end='\n\n')
print(study.best_params)

0.6946469105913182

{'bootstrap_type': 'Bernoulli', 'od_wait': 850, 'learning_rate': 0.08273393057193619, 'reg_lambda': 99.74069862420819, 'random_strength': 22.387576131424858, 'min_data_in_leaf': 13, 'leaf_estimation_iterations': 11, 'one_hot_max_size': 1}


In [ ]:
rskfold = RepeatedStratifiedKFold(n_splits=5, n_repeats=3)
clf = catboost.CatBoostClassifier(**study.best_params, verbose=0, task_type="GPU")

f1_list = []
for fold, (train_index, val_index) in enumerate(tqdm(rskfold.split(train_x, train_y))):
    x_trn, x_val, y_trn, y_val = train_x.loc[train_index], train_x.loc[val_index], train_y.loc[train_index], train_y.loc[val_index]

    clf.fit(x_trn, y_trn, early_stopping_rounds=100)
    f1_list.append(f1_score(clf.predict(x_val), y_val, average='macro'))

print(f'{min(f1_list)} ~ {max(f1_list)}')
print('mean :', np.mean(f1_list))

In [ ]:
rskfold = RepeatedStratifiedKFold(n_splits=5, n_repeats=3)
clf = catboost.CatBoostRegressor(**study.best_params, verbose=0, task_type="GPU")

f1_list = []
for fold, (train_index, val_index) in enumerate(tqdm(rskfold.split(train_x, train_yy))):
  x_trn, x_val, yy_trn, yy_val = train_x.loc[train_index], train_x.loc[val_index], train_yy.loc[train_index], train_yy.loc[val_index]
  y_trn = train_y.loc[train_index]
  y_val = train_y.loc[val_index]

  clf.fit(x_trn, y_trn, early_stopping_rounds=100, cat_features=['PRODUCT_CODE', 'LINE'])
  pre_preds = clf.predict(x_val)

  preds=[]
  for p in pre_preds:
    if p<=a[('Y_Quality','max')][0]:
      preds.append(0)
    elif p<=a[('Y_Quality','min')][2]:
      preds.append(1)
    else:
      preds.append(2)
  f1_list.append(f1_score(yy_val, preds, average='macro' ))

print(f'{min(f1_list)} ~ {max(f1_list)}')
print('mean :', np.mean(f1_list))

15it [12:00, 48.02s/it]

0.5355617060535094 ~ 0.7367773223539258
mean : 0.6547787687169611
